# Получение оценок из старого аккаунта

In [1]:
import math
import time
import re
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains

with webdriver.Chrome(service=Service(ChromeDriverManager().install())) as driver:
    driver.get('https://www.kinopoisk.ru/#login')

    # вхожу в аккаунт, из которого хочу перенести оценки и любимые фильмы
    # выделяю 40 секунд для авторизации и ввода капчи вручную.

    time.sleep(40)

    # после входа в систему определяю user_id
    user_icon_xpath = '//div[@class="styles_root__JgwJ_ styles_lg__KwIl6"][@data-tid="caacc44c"]'
    user_xpath = '//span[@class="styles_primaryTitle__lGNUB styles_primaryTitleDefaultAccount__a0_6V"] \
                 [@data-tid="ecca3393"]'
    user_icon = driver.find_element(By.XPATH, user_icon_xpath)
    ActionChains(driver).move_to_element(user_icon) \
                        .move_to_element(driver.find_element(By.XPATH, user_xpath)) \
                        .click().perform()
    user_id = re.search(r'\d+', driver.current_url)[0]

    # захожу на первую страницу с оценками, считаю количество страниц, по которым надо будет итерироваться

    ratings_page = 'https://www.kinopoisk.ru/user/' + user_id + \
                   '/votes/list/ord/date/page/{}/#list'
    driver.get(ratings_page.format(1))
    num_movies_selector = 'tbody > tr > td > h2'
    num_movies = driver.find_element(By.CSS_SELECTOR, num_movies_selector).text.split()[2][1:-1]
    num_pages = math.ceil(int(num_movies) / 50)

    # скачиваю просмотренные фильмы и выставленные им оценки

    movie_link_selector = 'div.nameRus > a'
    movie_rating_selector = 'div.selects.vote_widget span'
    all_links = []
    all_ratings = []

    for i in range(1, num_pages + 1):
        driver.get(ratings_page.format(i))
        movie_links = driver.find_elements(By.CSS_SELECTOR, movie_link_selector)
        ratings = driver.find_elements(By.CSS_SELECTOR, movie_rating_selector)
        all_links.extend([link.get_attribute('href') for link in movie_links])
        all_ratings.extend([item.text for item in ratings])
    all_ratings = [rate for rate in all_ratings if rate != '']

# Проставление полученных оценок в новом аккаунте

In [4]:
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as es
from selenium.common.exceptions import TimeoutException

with webdriver.Chrome(service=Service(ChromeDriverManager().install())) as driver:
    driver.get('https://www.kinopoisk.ru/#login')

    # выделяю 40 секунд для авторизации и ввода капчи вручную

    time.sleep(40)

    # вхожу в аккаунт, в который хочу перенести оценки и любимые фильмы
    # проставляю оценки all_ratings фильмам по ссылкам из списка all_links
    grade_button_xpath = '//div[@class="style_root__tg2Nx"][@data-tid="410c06ef"]'
    change_grade_button_xpath = '//span[@class="style_text__1QpiT"][@data-tid="e29193ab"]'

    for page, rating in zip(reversed(all_links), reversed(all_ratings)):
        driver.get(page)
        star_button_xpath = f'//div[@data-tid="48af2382"]/button[{rating}]'
        try:
            grade_button = WebDriverWait(driver, 7).until(
                            es.presence_of_element_located(
                                (By.XPATH, grade_button_xpath)))
            grade_button.click()
        except TimeoutException:
            raise TimeoutException(f'Dont find grade_button {page}')
        try:
            change_grade_button = WebDriverWait(driver, 1).until(
                                    es.presence_of_element_located(
                                        (By.XPATH, change_grade_button_xpath)))
            change_grade_button.click()
        except TimeoutException:
            pass
        try:
            star_button = WebDriverWait(driver, 1).until(
                            es.presence_of_element_located(
                                (By.XPATH, star_button_xpath)))
            webdriver.ActionChains(driver).move_to_element(star_button).click(star_button).perform()
        except TimeoutException:
            raise TimeoutException(f'Failed to rate movie {page}')